# A pair of Goodwin oscillators

Trying this instead of the thalamic model so that debugging isn't so goddamn slow.

# Imports

In [1]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
# user-defined
import nmCoupling as nm
import response as rp

import gw2

#from generate_figures import bif1d

import importlib
import lib

import lib.util
import lib.rhs

importlib.reload(sys.modules['lib.util'])
importlib.reload(sys.modules['lib.rhs'])
from lib.util import (get_phase, freq_est, get_tongue_1d,
                      pl_exist_1d)
from lib.rhs import (rhs_avg_2d, _redu_moving, 
                     _redu_moving_avg, rhs_avg_1d, rhs_avg_1dc)


from importlib import reload
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sympy as sym

from scipy.optimize import root,bisect
from copy import deepcopy

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

reload(gw2)
reload(nm)
reload(rp)

nmc = nm.nmCoupling
rsp = rp.Response

# Dicts

In [2]:
kw_sim = {'rtol':1e-7,'atol':1e-7,'method':'LSODA'}

In [3]:
pd1 = {'v1':.84,'v2':.42,'v4':.35,'v6':.35,'v8':1,
           'k1':1,'k2':1,'k3':.7,'k4':1,'k5':.7,
           'k6':1,'k7':.35,'k8':1,'K':0.5,'kc':1,
           'n':6,'L':0,'eps':0,'om':1,'om_fix':1}

In [4]:
kws1 = {'var_names':['x','y','z','v'],
        'pardict':pd1,
        'rhs':gw2.rhs,
        'coupling':gw2.coupling,
        'init':np.array([.3882,.523,1.357,.4347,24.2]),
        'TN':2000,
        'trunc_order':2,
        'z_forward':False,
        'i_forward':False,
        'i_bad_dx':[False,True,False,False],
        'max_iter':20,
        'rtol':1e-12,
        'atol':1e-12,
        'rel_tol':1e-9,
        'save_fig':False}

# Defs

In [5]:
x_temp = np.linspace(-np.pi,3*np.pi,500)

In [6]:
def bif1d(a,eps,del1):
    """
    for coupling only
    """

    y = rhs_avg_1dc(0,x_temp,a,eps,del1)

    #fig,axs = plt.subplots()
    #axs.plot(y)
    #plt.show()
    #plt.close()
    
    # get all zeros
    z1 = x_temp[1:][(y[1:]>0)*(y[:-1]<=0)]
    z2 = x_temp[1:][(y[1:]<0)*(y[:-1]>=0)]

    return z1,z2

In [7]:
def _full(t,y,a,eps,del1):
    pd1 = a.system1.pardict;pd2 = a.system2.pardict
    y1 = y[:4];y2 = y[4:]
    out1 = gw2.rhs(t,y1,pd1,'val',0) + eps*(del1 + gw2.coupling(y,pd1,'val',0))
    out2 = gw2.rhs(t,y2,pd2,'val',1) + eps*gw2.coupling(list(y2)+list(y1),pd2,'val',1)
    return np.array(list(out1)+list(out2))

In [8]:
def _redu(t,y,a,eps,del1):
    n = a._n[1];m = a._m[1]
    system1 = a.system1; system2 = a.system2
    h = 0
    for i in range(system1.miter):
        h += eps**(i+1)*(4*del1*np.mean(system1.z['dat'][i]) + system1.h['lam'][i](-y) - system2.h['lam'][i](-y))
    return h*n

In [9]:
# def _redu2(t,y,a,eps,del1):
#     n = a._n[1];m = a._m[1]
#     system1 = a.system1; system2 = a.system2
#     h = 0
#     for i in range(system1.miter):
#         h += eps**(i+1)*(4*del1*np.mean(system1.z['dat'][i]) + system1.h['lam'][i](y) - system2.h['lam'][i](y))
#     return h*n/m

In [10]:
def add_diagram(axs,a,del1,eps_list):
    zu_list = []
    zs_list = []

    zu = []
    zs = []

    for eps in eps_list:
        z1,z2 = bif1d(a,eps,del1)

        zu.append(np.mod(z1,2*np.pi))
        zs.append(np.mod(z2,2*np.pi))

    for xe, ye in zip(eps_list, zu):
        axs.scatter([xe] * len(ye), ye,s=2,c='red')

    for xe, ye in zip(eps_list, zs):
        axs.scatter([xe] * len(ye), ye,s=2,c='k')

    axs.set_ylim(-.1,2*np.pi+.1)
    axs.set_xlim(eps_list[0],eps_list[-1])
    

# Initialize

In [11]:
system1 = rsp(idx=0,model_name='gw0',**kws1)
system2 = rsp(idx=1,model_name='gw1',**kws1)

* Loading LC data...
self.T 6.283185307179586 omfix 3.8590183230656914
* Loading monodromy...
* Floquet Exponent kappa =-0.15846659975461394
* Loading g symbolic...
* Computing g...
* Loading heterogeneous terms...
* Computing z...
* Computing i...
* Loading LC data...
self.T 6.283185307179586 omfix 3.8590183230656914
* Loading monodromy...
* Floquet Exponent kappa =-0.15846659975461394
* Loading g symbolic...
* Computing g...
* Loading heterogeneous terms...
* Computing z...
* Computing i...


# 1:1

In [ ]:
a11 = nm.nmCoupling(system1,system2,
                    recompute_list=['p_data_gw0','p_data_gw1','h_data_gw0','h_data_gw1'],
                    #recompute_list=['h_data_gw0','h_data_gw1'],
                    #recompute_list=recompute_list,
                    _n=('om0',1),_m=('om1',1),
                    NP=500,
                    NH=500,
                    save_fig=True)

* Loading p symbolic...
* Loading p symbolic...
p order=0
p order=0
p order=1
p order=1
p order=2


In [ ]:
eps11 = .3
del11 = -0.02
th_init11 = 3

## phase plane

In [ ]:
x_temp2 = np.linspace(0,2*np.pi,200)

rhs = _redu(0,x_temp2,a11,eps11,del11)

fig,axs = plt.subplots(figsize=(6,2))
axs.plot(x_temp2,rhs)
axs.axhline(0,0,2*np.pi,ls='--',color='gray')

## Integrate

In [ ]:
dt = .01;T=200
t11 = np.arange(0,T,dt)
args11 = {'args':(a11,eps11,del11),'t_eval':t11,**kw_sim}

In [ ]:
y0a = a11.system1.lc['dat'][int((th_init11/(2*np.pi))*a11.system1.TN),:]
y0b = a11.system2.lc['dat'][int((0/(2*np.pi))*a11.system2.TN),:]

y0 = np.array([y0a,y0b]).flatten()

solf11 = solve_ivp(_full,[0,t11[-1]],y0,**args11)

In [ ]:
solr11 = solve_ivp(_redu2,[0,t11[-1]],[th_init11],**args11)

In [ ]:
fig,axs = plt.subplots(figsize=(10,4))
axs.plot(solf11.t,solf11.y.T[:,0])
axs.plot(solf11.t,solf11.y.T[:,4])
#axs.set_xlim(sol_full.t[-1]-200,sol_full.t[-1])
axs.set_xlim(t11[-1]-400,t11[-1])

In [ ]:
tp11a, phase11a = get_phase(t11,solf11.y.T[:,:4],skipn=20,system1=a11.system1)
tp11b, phase11b = get_phase(t11,solf11.y.T[:,4:],skipn=20,system1=a11.system2)

## Full vs phase

In [ ]:
fig,axs = plt.subplots()
y = np.mod(phase11a - a11.om*phase11b,2*np.pi)
axs.scatter(tp11a,y,s=1,alpha=.5)
axs.plot(t11,np.mod(solr11.y.T,2*np.pi),color='tab:red')
#axs.scatter(sol_full.t[::skipn],np.mod(phase2,1),s=1)
axs.set_ylim(0,2*np.pi)

## Bifurcations

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(10,2))

del_list11 = np.linspace(-.02,.02,len(axs))
eps_list11 = np.linspace(.001,.4,30)

for k in range(len(del_list11)):
    add_diagram(axs[k],a11,del_list11[k],eps_list11)

# 1:2

In [ ]:
a12 = nm.nmCoupling(system1,system2,
                    #recompute_list=['p_data_gw0','p_data_gw1','h_data_gw0','h_data_gw1'],
                    #recompute_list=['h_data_gw0','h_data_gw1'],
                    #recompute_list=recompute_list,
                    _n=('om0',1),_m=('om1',2),
                    NP=700,
                    NH=700,
                    save_fig=True)

In [ ]:
eps12 = .12
del12 = 0.0
th_init12 = 1.75

First order starts losing accuracy around eps = 0.025 as eps increases (the reduced model reaches the same fixed point as the full model). Even at eps=0.02, the reduced and full models don't coincide in transients despite the long-term convergence.

Using order eps^2 at eps = 0.025 results in WORSE performance... So clearly if not p then there is something wrong with calculating h functions...

## Phase plane

In [ ]:
x_temp2 = np.linspace(0,2*np.pi,200)

# def _redu(t,y,a,eps,del1):
#     n = a._n[1];m = a._m[1]
#     system1 = a.system1; system2 = a.system2
#     h = 0
#     for i in range(system1.miter):
#         h += eps**(i+1)*(4*del1*np.mean(system1.z['dat'][i]) + system1.h['lam'][i](-y+np.pi/4) - system2.h['lam'][i](-y+np.pi/4))
#     return h*n/m

rhs = _redu(0,x_temp2,a12,eps12,del12)

fig,axs = plt.subplots()
axs.plot(x_temp2,rhs)
axs.axhline(0,0,2*np.pi,ls='--',color='gray')

## Integrate

In [ ]:
dt = .01;T=1000
t12 = np.arange(0,T,dt)
args12 = {'args':(a12,eps12,del12),'t_eval':t12,**kw_sim}

In [ ]:
y0a = a12.system1.lc['dat'][int((th_init12/(2*np.pi))*a12.system1.TN),:]
y0b = a12.system2.lc['dat'][int((0/(2*np.pi))*a12.system2.TN),:]

y0 = np.array([y0a,y0b]).flatten()

solf12 = solve_ivp(_full,[0,t12[-1]],y0,**args12)

In [ ]:
solr12 = solve_ivp(_redu,[0,t12[-1]],[th_init12],**args12)

In [ ]:
fig,axs = plt.subplots(2,1,figsize=(10,4))
axs[0].plot(solf12.t,solf12.y.T[:,0])
axs[0].plot(solf12.t,solf12.y.T[:,4])

axs[1].plot(solf12.t,solf12.y.T[:,0])
axs[1].plot(solf12.t,solf12.y.T[:,4])

axs[0].set_xlim(0,200)
axs[1].set_xlim(t12[-1]-200,t12[-1])

In [ ]:
tp12a, phase12a = get_phase(t12,solf12.y.T[:,:4],skipn=50,system1=a12.system1)
tp12b, phase12b = get_phase(t12,solf12.y.T[:,4:],skipn=50,system1=a12.system2)

## Full vs phase

In [ ]:
fig,axs = plt.subplots()
y = np.mod(phase12a - a12.om*phase12b,2*np.pi)
axs.scatter(tp12a,y,s=1,alpha=.5)
axs.plot(t12,np.mod(solr12.y.T,2*np.pi),color='tab:red')
#axs.scatter(sol_full.t[::skipn],np.mod(phase2,1),s=1)
axs.set_ylim(0,2*np.pi)

### Multiple Initial Conditions

In [ ]:
init_phases12 = np.linspace(0,2*np.pi,10,endpoint=False)
dt = .01;T=1000
t12b = np.arange(0,T,dt)

y0b = a12.system2.lc['dat'][int((0/(2*np.pi))*a12.system2.TN),:]

full_1 = []
full_2 = []

args12b = {'args':(a12,eps12,del12),'t_eval':t12b,**kw_sim}

for i in range(len(init_phases12)):
    y0a = a12.system1.lc['dat'][int((init_phases12[i]/(2*np.pi))*a12.system1.TN),:]
    y0 = np.array([y0a,y0b]).flatten()

    solf12 = solve_ivp(_full,[0,t12b[-1]],y0,**args12b)
    tp12a, phase12a = get_phase(t12b,solf12.y.T[:,:4],skipn=50,system1=a12.system1)
    tp12b, phase12b = get_phase(t12b,solf12.y.T[:,4:],skipn=50,system1=a12.system2)

    full_1.append([tp12a,phase12a])
    full_2.append([tp12b,phase12b])

In [ ]:
redu_1 = []

for i in range(len(init_phases12)):
    solf12 = solve_ivp(_redu,[0,t12b[-1]],[init_phases12[i]],**args12b)
    redu_1.append(solf12)

In [ ]:
fig,axs = plt.subplots(2,1)
axs[0].plot(x_temp2,rhs)
axs[0].axhline(0,0,2*np.pi,ls='--',color='gray')

for i in range(len(init_phases12)):
    y = np.mod(full_1[i][1] - a12.om*full_2[i][1],2*np.pi)
    axs[1].scatter(y,full_1[i][0],s=1,alpha=.5,color='k')
    axs[1].scatter(np.mod(redu_1[i].y[0],2*np.pi),redu_1[i].t,color='r',s=1,zorder=10)
    
axs[0].set_xlim(0,2*np.pi)
axs[1].set_xlim(0,2*np.pi)

## Bifurcations

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(10,2))

del_list12 = np.linspace(-.002,.002,3)
eps_list12 = np.linspace(0.001,.25,50)

for k in range(len(del_list12)):
    add_diagram(axs[k],a12,del_list12[k],eps_list12)

## Unaveraged

Compare simulations between full and reduced using non-averaged, original phase-isostable coordinates.

### Z,I functions and rhs

The psi/theta equations aren't implemented in the coupling case, so I'll do it here first.

I can't use `system1.K['sym']` or `system1.G['sym']` because they are both in terms of epsilon. I need to fall back to `system1.z['vec_psi']` and use `system1.z_og` to define a substitution rule. `a12.z_og` contains indexed variables used to define the $\psi$ expansion. 

`system1.z_og` is in the order: `zx0[0], zx0[1], zy0[0], zy0[1]`, i.e., variable 1, all orders, then next variable all orders, etc.

In [ ]:
print(a12.system1.z_og)

In [ ]:
print(a12.system1.i['vec_psi'])
print(a12.system1.z['vec_psi'])

In [ ]:
rule_var2imp = {}

# system 1
count = 0
for vname in a12.system1.var_names:
    for k in range(a12.system1.miter):
        rule_var2imp[a12.system1.z_og[count]] = a12.system1.z['imp_'+vname][k](a12.ths[0])
        rule_var2imp[a12.system1.i_og[count]] = a12.system1.i['imp_'+vname][k](a12.ths[0])
        rule_var2imp[a12.system1.g_og[count]] = a12.system1.g['imp_'+vname][k](a12.ths[0])
        count += 1
        
    #rule_var2imp[a12.system1.lc_og] = a12.system1.lc['imp_'+vname](a12.ths[0])

# system 2
count = 0
for vname in a12.system2.var_names:
    for k in range(a12.system2.miter):
        rule_var2imp[a12.system2.z_og[count]] = a12.system2.z['imp_'+vname][k](a12.ths[1])
        rule_var2imp[a12.system2.i_og[count]] = a12.system2.i['imp_'+vname][k](a12.ths[1])
        rule_var2imp[a12.system2.g_og[count]] = a12.system2.g['imp_'+vname][k](a12.ths[1])
        count += 1
        
    #rule_var2imp[a12.system2.lc_og] = a12.system2.lc['imp_'+vname](a12.ths[1])


print(rule_var2imp)

Now that I have the implemented functions, it's time to substitute out the symbols like `zx[0]` with the corresponding implemented function, then lambdify.

In [ ]:
print(a12.system1.i['vec_psi'].subs(rule_var2imp))
print(a12.system2.z['vec_psi'].subs(rule_var2imp))

In [ ]:
i_lam0 = sym.lambdify([a12.ths[0],a12.system1.psi],a12.system1.i['vec_psi'].subs(rule_var2imp).T)
z_lam0 = sym.lambdify([a12.ths[0],a12.system1.psi],a12.system1.z['vec_psi'].subs(rule_var2imp).T)

i_lam1 = sym.lambdify([a12.ths[1],a12.system2.psi],a12.system2.i['vec_psi'].subs(rule_var2imp).T)
z_lam1 = sym.lambdify([a12.ths[1],a12.system2.psi],a12.system2.z['vec_psi'].subs(rule_var2imp).T)

In [ ]:
print(a12.system1.var_names)

Define lambidfied functions for use in unaveraged rhs

In [ ]:
psi_sym = a12.system1.psi
imp = a12.system1.lc['imp_x0'](a12.ths[0])+psi_sym*a12.system1.g['imp_x0'][1](a12.ths[0])+psi_sym**2*a12.system1.g['imp_x0'][2](a12.ths[0])
x0_lam = sym.lambdify([a12.ths[0],psi_sym],imp)

imp = a12.system1.lc['imp_y0'](a12.ths[0])+psi_sym*a12.system1.g['imp_y0'][1](a12.ths[0])+psi_sym**2*a12.system1.g['imp_y0'][2](a12.ths[0])
y0_lam = sym.lambdify([a12.ths[0],psi_sym],imp)

imp = a12.system1.lc['imp_z0'](a12.ths[0])+psi_sym*a12.system1.g['imp_z0'][1](a12.ths[0])+psi_sym**2*a12.system1.g['imp_z0'][2](a12.ths[0])
z0_lam = sym.lambdify([a12.ths[0],psi_sym],imp)

imp = a12.system1.lc['imp_v0'](a12.ths[0])+psi_sym*a12.system1.g['imp_v0'][1](a12.ths[0])+psi_sym**2*a12.system1.g['imp_v0'][2](a12.ths[0])
v0_lam = sym.lambdify([a12.ths[0],psi_sym],imp)

Unaveraged RHS

In [ ]:
pd1 = a12.system1.pardict;pd2 = a12.system2.pardict
def _redu_c(t,y,eps=0,del1=0):
    """full reduced model"""
    th0,ps0,th1,ps1 = y
    
    x0 = x0_lam(th0,ps0);y0 = y0_lam(th0,ps0);z0 = z0_lam(th0,ps0);v0 = v0_lam(th0,ps0)
    x1 = x0_lam(th1,ps1);y1 = y0_lam(th1,ps1);z1 = z0_lam(th1,ps1);v1 = v0_lam(th1,ps1)

    yin0 = [x0,y0,z0,v0,x1,y1,z1,v1]
    c0a = z_lam0(th0,ps0)[0,0]*gw2.coupling(yin0,pd1,'val',0)[0]
    c0b = i_lam0(th0,ps0)[0,0]*gw2.coupling(yin0,pd1,'val',0)[0]
    
    dth0 = 1 + eps*(del1 + c0a)
    dps0 = a12.system1.kappa_val*ps0 + eps*c0b

    yin1 = [x1,y1,z1,v1,x0,y0,z0,v0]
    c1a = z_lam1(th1,ps1)[0,0]*gw2.coupling(yin1,pd1,'val',0)[0]
    c1b = i_lam1(th1,ps1)[0,0]*gw2.coupling(yin1,pd1,'val',0)[0]
    
    dth1 = 1 + eps*c1a
    dps1 = a12.system2.kappa_val*ps1 + eps*c1b

    return np.array([dth0,dps0,dth1*a12._m[1],dps1*a12._m[1]])

### Integrate

In [ ]:
t = np.arange(0,1000,.01)
init = [2.5,.0,0,.0]
out = solve_ivp(_redu_c,[t[0],t[-1]],init,args=(eps12,del12),t_eval=t,**kw_sim)

In [ ]:
fig,axs = plt.subplots()
axs.plot(t,out.y[0]-a12.om*out.y[2],alpha=.75)
#axs.plot(t,out.y[1])
y = np.mod(phase12a - a12.om*phase12b,2*np.pi)
axs.scatter(tp12a,y,s=1,alpha=.5,color='k')
#axs.plot(t12,np.mod(solr12.y.T,2*np.pi),color='tab:red')
#axs.plot(t,out.y[3])
#axs.plot(t,np.mod(out.y[0],2*np.pi))
#axs.plot(t,np.mod(out.y[2],2*np.pi))
axs.set_ylim(-.1,2*np.pi+.1)
#axs.set_xlim(0,10)

### Multiple Initial Conditions

In [ ]:
init_phases12b = np.linspace(0,2*np.pi,5,endpoint=False)

In [ ]:
phase_iso = []
t = np.arange(0,1000,.01)
for i in range(len(init_phases12b)):
    init = [init_phases12b[i],.0,0,.0]
    solr12 = solve_ivp(_redu_c,[0,t12b[-1]],init,args=(eps12,del12),t_eval=t,**kw_sim)
    phase_iso.append(solr12)

In [ ]:
fig,axs = plt.subplots(2,1)
axs[0].plot(x_temp2,rhs)
axs[0].axhline(0,0,2*np.pi,ls='--',color='gray')

skipn = 1
for i in range(len(init_phases12)):
    y = np.mod(full_1[i][1] - a12.om*full_2[i][1],2*np.pi)
    axs[1].scatter(y[::skipn],full_1[i][0][::skipn],s=1,alpha=.5,color='k')
    axs[1].scatter(np.mod(redu_1[i].y[0]+np.pi/4,2*np.pi),redu_1[i].t,color='r',s=1,zorder=10)

skipn = 100
for i in range(len(init_phases12b)):
    y2 = np.mod(phase_iso[i].y[0] - a12.om*phase_iso[i].y[2],2*np.pi)
    axs[1].scatter(y2[::skipn],phase_iso[i].t[::skipn],color='b',s=1,zorder=20,alpha=.5)
    
#axs[0].set_xlim(0,2*np.pi)
#axs[1].set_xlim(0,2*np.pi)
plt.savefig('unave_blue_vs_full_black_vs_1d_red.png')

## Check phase indexing

In [ ]:
# check indexing for phase
sol_arr = solf12.y.T[:2000,:4]

fig,axs = plt.subplots()
axs.plot(sol_arr[:,0])

phase_value = []


for i in range(len(sol_arr)):
    d1 = np.linalg.norm(sol_arr[i,:]-a12.system1.lc['dat'],axis=1)
    phase_value.append(np.argmin(d1)/len(system1.lc['dat']))

axs.plot(phase_value)

# 2:1

In [ ]:
a21 = nm.nmCoupling(system1,system2,
                    recompute_list=['p_data_gw0','p_data_gw1','h_data_gw0','h_data_gw1'],
                    #recompute_list=recompute_list,
                    _n=('om0',2),_m=('om1',1),
                    NP=700,
                    NH=700,
                    save_fig=True)

In [ ]:
eps21 = .05
del21 = 0.0
th_init21 = 0

First order starts losing accuracy around eps = 0.025 as eps increases (the reduced model reaches the same fixed point as the full model). Even at eps=0.02, the reduced and full models don't coincide in transients despite the long-term convergence.

Using order eps^2 at eps = 0.025 results in WORSE performance... So clearly if not p then there is something wrong with calculating h functions...

## Phase plane

In [ ]:
x_temp2 = np.linspace(0,4*np.pi,200)

rhs = _redu(0,x_temp2,a21,eps21,del21)

fig,axs = plt.subplots()
axs.plot(x_temp2,rhs)
axs.axhline(0,0,2*np.pi,ls='--',color='gray')

## Integrate

In [ ]:
dt = .01;T=4000
t21 = np.arange(0,T,dt)
args21 = {'args':(a21,eps21,del21),'t_eval':t21,**kw_sim}

In [ ]:
y0a = a21.system1.lc['dat'][int((th_init21/(2*np.pi))*a21.system1.TN),:]
y0b = a21.system2.lc['dat'][int((0/(2*np.pi))*a21.system2.TN),:]

y0 = np.array([y0a,y0b]).flatten()

solf21 = solve_ivp(_full,[0,t21[-1]],y0,**args21)

In [ ]:
solr21 = solve_ivp(_redu,[0,t21[-1]],[th_init21],**args21)

In [ ]:
fig,axs = plt.subplots(figsize=(10,4))
axs.plot(solf21.t,solf21.y.T[:,0])
axs.plot(solf21.t,solf21.y.T[:,4])
#axs.set_xlim(sol_full.t[-1]-200,sol_full.t[-1])
axs.set_xlim(t21[-1]-200,t21[-1])

In [ ]:
tp21a, phase21a = get_phase(t21,solf21.y.T[:,:4],skipn=50,system1=a21.system1)
tp21b, phase21b = get_phase(t21,solf21.y.T[:,4:],skipn=50,system1=a21.system2)

## Full vs phase

In [ ]:
fig,axs = plt.subplots()
y = np.mod(phase21a - a21.om*phase21b,2*np.pi)
axs.scatter(tp21a,y,s=1,alpha=.5)
axs.plot(t21,np.mod(solr21.y.T,2*np.pi),color='tab:red')
#axs.scatter(sol_full.t[::skipn],np.mod(phase2,1),s=1)
axs.set_ylim(0,2*np.pi)

## Bifurcations

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(10,2))

del_list21 = np.linspace(-.002,.002,3)
eps_list21 = np.linspace(0.001,.25,50)

for k in range(len(del_list21)):
    add_diagram(axs[k],a21,del_list21[k],eps_list21)